In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

### Preprocessing

In [40]:
# load data
mnist_dataset, mnist_info = tfds.load(name='mnist', as_supervised=True, with_info = True)

# train-test already split
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# validation will be 10% of train
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# number of tests
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# scale each image from 0-1
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.0
  return image, label

# scale data
scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_and_validation_data = mnist_test.map(scale)

# shuffle for batches
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


# 100 batches
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

# validation/test does not need batches, but tf expects batches, so create one batch
validation_data = validation_data.batch(num_validation_samples)
test_data = mnist_test.batch(num_test_samples)

# split inputs and targets: there is one batch so next(iter()) returns the first batch
validation_inputs, validation_targets = next(iter(validation_data))

# test_inputs, test_targets = next(iter(test_data))

### Creating the model

In [41]:
input_size = 784
output_size = 10
hidden_layer_size = 200


model = tf.keras.Sequential([
       tf.keras.layers.Flatten(input_shape=(28,28,1)),
       tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
       tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
       tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
       tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
       tf.keras.layers.Dense(output_size, activation='softmax'),
])

In [42]:
# sparse_categorical_crossentropy applies one-hot encoding 
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [43]:
NUM_EPOCHS = 10

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/10
540/540 - 9s - loss: 0.2550 - accuracy: 0.9218 - val_loss: 0.1277 - val_accuracy: 0.9605
Epoch 2/10
540/540 - 6s - loss: 0.1026 - accuracy: 0.9679 - val_loss: 0.0759 - val_accuracy: 0.9788
Epoch 3/10
540/540 - 6s - loss: 0.0685 - accuracy: 0.9790 - val_loss: 0.0618 - val_accuracy: 0.9820
Epoch 4/10
540/540 - 6s - loss: 0.0541 - accuracy: 0.9832 - val_loss: 0.0586 - val_accuracy: 0.9812
Epoch 5/10
540/540 - 6s - loss: 0.0450 - accuracy: 0.9855 - val_loss: 0.0629 - val_accuracy: 0.9825
Epoch 6/10
540/540 - 6s - loss: 0.0394 - accuracy: 0.9873 - val_loss: 0.0389 - val_accuracy: 0.9877
Epoch 7/10
540/540 - 6s - loss: 0.0310 - accuracy: 0.9898 - val_loss: 0.0306 - val_accuracy: 0.9913
Epoch 8/10
540/540 - 6s - loss: 0.0303 - accuracy: 0.9901 - val_loss: 0.0319 - val_accuracy: 0.9905
Epoch 9/10
540/540 - 6s - loss: 0.0245 - accuracy: 0.9919 - val_loss: 0.0371 - val_accuracy: 0.9887
Epoch 10/10
540/540 - 6s - loss: 0.0257 - accuracy: 0.9921 - val_loss: 0.0332 - val_accuracy: 0.9888

### Testing

In [44]:
model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.1218 - accuracy: 0.9779


[0.1218169555068016, 0.9779000282287598]